In [12]:
# %pip install tensorflow
# %pip freeze > requirements.txt

In [13]:
# generate training set
from tensorflow import keras

train_set = keras.preprocessing.image_dataset_from_directory(
	'../../input/images/training_testing/random-5500-0.8/training/',
	validation_split=0.2,
	subset="training",
	seed=123, #TODO: understand this arg
	image_size=(224, 224),
	batch_size=32,
)
train_set.class_names


Found 11000 files belonging to 2 classes.
Using 8800 files for training.


['no whale', 'whale present']

In [14]:
# generate validation set
validation_set = keras.preprocessing.image_dataset_from_directory(
	'../../input/images/training_testing/random-5500-0.8/training/',
	validation_split=0.2,
	subset="validation",
	seed=123, #TODO: understand this arg
	image_size=(224, 224),
	batch_size=32,
)

Found 11000 files belonging to 2 classes.
Using 2200 files for validation.


In [15]:
for images, labels in train_set:
    preprocessed_images = keras.applications.resnet50.preprocess_input(images)

preprocessed_images, labels


(<tf.Tensor: shape=(32, 224, 224, 3), dtype=float32, numpy=
 array([[[[ 6.0997009e-02, -4.2778999e+01,  9.9320000e+01],
          [ 6.0997009e-02, -4.2778999e+01,  9.9320000e+01],
          [-1.9390030e+00, -3.9778999e+01,  1.0232000e+02],
          ...,
          [-1.0939003e+01,  6.2210007e+00,  1.2532000e+02],
          [-1.1939003e+01, -1.5778999e+01,  1.1932000e+02],
          [-1.1939003e+01, -1.5778999e+01,  1.1932000e+02]],
 
         [[ 6.0997009e-02, -4.2778999e+01,  9.9320000e+01],
          [ 6.0997009e-02, -4.2778999e+01,  9.9320000e+01],
          [-1.9390030e+00, -3.9778999e+01,  1.0232000e+02],
          ...,
          [-1.0939003e+01,  6.2210007e+00,  1.2532000e+02],
          [-1.1939003e+01, -1.5778999e+01,  1.1932000e+02],
          [-1.1939003e+01, -1.5778999e+01,  1.1932000e+02]],
 
         [[ 6.0997009e-02, -4.2778999e+01,  9.9320000e+01],
          [ 6.0997009e-02, -4.2778999e+01,  9.9320000e+01],
          [-1.9390030e+00, -3.9778999e+01,  1.0232000e+02],
    

In [16]:
# load resnet50 model with pretrained weights
# replace output layer with 2 neurons
# connect to gpu?
# shuffle training set
#  cross entrpoy loss, otimize adam model parameters learning rate 0.0001 epochs 10
# save model
# confusion matrix

In [17]:
# base_model = keras.applications.ResNet50(
# 	weights= None, 
# 	# include_top=False, 
# 	# input_shape= (img_height,img_width,3)
# 	num_classes=2
# 	)

imported_model= keras.applications.ResNet50(include_top=False,
	input_shape=(224,224,3),
	pooling='avg',
	classes=2,
	# weights='imagenet'
	weights=None
)

In [18]:
custom_model = keras.models.Sequential()
custom_model.add(imported_model)
custom_model.add(keras.layers.Dense(2, activation='softmax'))
custom_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_1 (Dense)             (None, 2)                 4098      
                                                                 
Total params: 23591810 (90.00 MB)
Trainable params: 23538690 (89.79 MB)
Non-trainable params: 53120 (207.50 KB)
_________________________________________________________________


In [19]:
from keras.optimizers.legacy import Adam
custom_model.compile(
	optimizer=Adam(learning_rate=0.001),
	loss='sparse_categorical_crossentropy',
	metrics=['accuracy']
)

In [20]:
history = custom_model.fit(
	train_set,
	validation_data=validation_set,
	epochs=5
)

Epoch 1/5
129/275 [=============>................] - ETA: 6:51 - loss: 0.6767 - accuracy: 0.7493

In [ ]:
# save custom_model
custom_model.save('resnet50_epochs2_samples5500_seed123.h5')

/Users/lshuster/Library/CloudStorage/OneDrive-GlobalHealthcareExchange/Desktop/whale-detection/.conda/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
